<a href="https://colab.research.google.com/github/Sotrosca/perceptron_template/blob/master/perceptron_multicapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inicializacion de numpy y ABCMeta. Definicion de las funciones de activacion que usamos.

# Inicializacion de numpy y ABCMeta. Definicion de las funciones de activacion que usamos.

In [0]:
import numpy as np

In [0]:
from abc import ABCMeta, abstractmethod

class FuncionActivacion(metaclass=ABCMeta):
  def __init__(self, Nombre):
    self.nombre = Nombre
  
  def nombreFuncion(self):
    return self.nombre
  
  @abstractmethod
  def val(self, x):
    pass
  
  @abstractmethod
  def derivada(self, x):
    pass
  
  #faltaria agregar una funcion que deje calcular el error dependiendo de la funcion
  def cal_error(self, x, y):
    pass
  
class Escalon(FuncionActivacion):
  def __init__(self):
    super().__init__("escalon")
  
  def val(self, x):
    return np.where(x >= 0.0, 1, 0)
  
  def derivada(self, x):
    return 1

class TanHiperbolica(FuncionActivacion):
  def __init__(self):
    super().__init__("tan hiperbolica")
  
  def val(self, x):
    return np.tanh(x)
  
  def derivada(self, x):
    tanh = np.tanh(x)
    return (1 + tanh) * (1 - tanh)

class Logistica(FuncionActivacion):
  def __init__(self):
    super().__init__("logistica")
  
  def val(self, x):
    return 1 / (1 + np.e ** (-x))
  
  def derivada(self, x):
    logistica = self.val(x)
    return logistica * (1 - logistica)

#Define las clases capa y red

In [0]:
class Neural_Layer():
  def __init__(self, dimensionInput, cantidadNeuronas, funcionActivacion):
    self.dimensionInput = dimensionInput
    self.cantidadNeuronas = cantidadNeuronas
    self.funcionActivacion = funcionActivacion
    self.W_ = (np.random.rand(cantidadNeuronas, dimensionInput + 1) *2) - 1 #La primer columna corresponde al Bias que va a cada neurona de salida. / 
    
  def outputCapa(self, inputCapa):
    #print(inputCapa)
    return self.W_[:, 0] + self.W_[:, 1:] @ inputCapa
    
  def predictCapa(self, inputCapa):    
    outputCapa = self.outputCapa(inputCapa)
    return self.funcionActivacion.val(outputCapa)

In [0]:
class Neural_Net():
  def __init__(self, topology, listaFuncionesActivacion, nombre='null'):
    self.nombre = nombre
    self.numeroCapasOcultas = len(topology) - 1 #Incluye la capa de salida, excluye la de entrada.
    self.errors_= []
    self.errorsValidation_ = []
    self.capas = []
    for i in range(self.numeroCapasOcultas):
      capa = Neural_Layer(dimensionInput=topology[i], cantidadNeuronas=topology[i + 1], funcionActivacion=listaFuncionesActivacion[i])
      self.capas.append(capa)
      
  def predict(self, inputRed): #arma una lista con el input, y los resultados despues de pasar por cada capa de la red
    listaPredictsCapas = [inputRed]
    
    for i in range(self.numeroCapasOcultas):
      predict = self.capas[i].predictCapa(listaPredictsCapas[-1])
      listaPredictsCapas.append(predict)
    
    return listaPredictsCapas
  
  def train(self, conjuntoEntrenamiento, targetEntrenamiento, epochs=2000, lr=0.7):
    
    for i in range(epochs):
      errors = 0
      for xi, target in zip(conjuntoEntrenamiento, targetEntrenamiento):
        listaPredict_xi = self.predict(xi)

        predict_xi = listaPredict_xi[-1]

        errors += (np.sum((target - predict_xi) ** 2)) / 2
        listaDeltas = self.calculaDeltas(listaPredict_xi, target)

        for i in range(1, 1 + len(listaDeltas), 1):
          delta_W = lr * np.array([listaDeltas[i - 1]]).T @ np.array([listaPredict_xi[i - 1]])
          self.capas[i - 1].W_[:, 1:] = self.capas[i - 1].W_[:, 1:] - delta_W
          self.capas[i - 1].W_[:, 0] = self.capas[i - 1].W_[:, 0] - lr * listaDeltas[i - 1]
          

      self.errors_.append(errors)
      
  def calculaDeltas(self, listaPredicts, targetPredict):
    listaDeltas = []
    for i in range(len(listaPredicts) - 1, 0, -1):
      
      if i == len(listaPredicts) - 1:
        delta = self.capas[i - 1].funcionActivacion.derivada(self.capas[i - 1].outputCapa(listaPredicts[-2])) * (listaPredicts[-1] - targetPredict)
        listaDeltas.insert(0, delta)
        
      else:
        delta = self.capas[i - 1].funcionActivacion.derivada(self.capas[i - 1].outputCapa(listaPredicts[-2 + i -(len(listaPredicts) - 1)])) * (self.capas[i].W_[:, 1:].T @ listaDeltas[0])
        listaDeltas.insert(0, delta)
    return listaDeltas
   

#Ejemplos y pruebas

In [0]:
red = Neural_Net([4, 3, 5, 2, 1], [Logistica(), Logistica(), Logistica(), Logistica()])

#print(red.capas[0].W_[0,:])
#print("-------------------")
#print(red.capas[0].W_[1:,:])
#print("-------------------")
#print(np.array([1,1,1,1]))
#print("-------------------")
#print(Logistica().val(red.capas[0].W_[0,:] + [1,1,1,1] @ red.capas[0].W_[1:,:]))
#print(red.predict([0.5,0.5,0.5,0.5])[1].shape)
#print(red.capas[0].W_)
#print(red.calculaDeltas(red.predict([0, 0, 0, 0]), 1))
#a = red.train(np.array([[1,1,1,1], [2,2,2,2]]), [0, 1])
#predict = red.predict([1, 1, 1, 1])

#print(predict)
#print(red.predict([1, 1, 1, 0]))
#print(red.predict([1, 1, 0, 1]))
#print(red.predict([1, 0, 1, 1]))
#print(red.predict([0, 1, 1, 1]))
#print(red.predict([0, 0, 0, 1]))
#print(red.predict([0, 0, 0, 0]))

In [0]:
for i, capa in enumerate(red.capas):
  print(i)
  print("pesos" + str(capa.W_))

predict1111 = red.capas[0].predictCapa([1,1,1,1])
predict0000 = red.capas[0].predictCapa([-10,-10,-10,-10])

print(red.capas[0].outputCapa([-1,-1,-1,-1]))
print(red.capas[0].predictCapa([1,1,1,1]))
print("-----")
print(red.capas[1].outputCapa(predict1111))
print(red.capas[1].predictCapa(predict1111))
print("-----")
print(red.capas[0].outputCapa([0,0,0,0]))
print(red.capas[0].predictCapa([0,0,0,0]))
print("-----")
print(red.capas[1].outputCapa(predict0000))
print(red.capas[1].predictCapa(predict0000))

0
pesos[[-0.16369542  0.24762822 -0.17075841  0.23762158 -0.22394278]
 [ 0.22294288  0.23765382  0.2136152  -0.15414583  0.16740243]
 [ 0.15303879  0.13285885 -0.12538215  0.22832843  0.0854131 ]]
1
pesos[[ 0.12301973 -0.13590302  0.01921756  0.10449247]
 [-0.2085546   0.23718288 -0.24640626 -0.08047607]
 [ 0.11739907 -0.02726077 -0.22101341 -0.08416231]
 [-0.08719602  0.24497108 -0.07293514  0.00406363]
 [-0.15145855  0.12313932  0.13316067  0.1489683 ]]
2
pesos[[ 0.00962114 -0.11938549  0.14919571  0.06811921 -0.05960127 -0.13638949]
 [-0.18970498 -0.10720422 -0.20103466  0.05877213  0.06964012 -0.21413925]]
3
pesos[[ 0.17740268  0.17776585 -0.05215858]]
[-0.25424403 -0.24158275 -0.16817945]
[0.48172144 0.66540355 0.61639084]
-----
[ 0.13474797 -0.30786284 -0.09467301 -0.01521472  0.08828858]
[0.53363611 0.42363648 0.47634941 0.49619639 0.52205782]
-----
[-0.16369542  0.22294288  0.15303879]
[0.45916728 0.55550601 0.5381852 ]
-----
[ 0.09320033 -0.15447108  0.10403772 -0.02527466 -0.

In [0]:
capa1 = red.capas[1].W_
bias = red.capas[1].W_[:, 0]
pesos = red.capas[1].W_[:, 1:]
print(pesos)
print(predict1111)
print(predict0000)
print(pesos @ predict1111)
print(pesos @ predict0000)

In [0]:
print([-186847283902439/2500000000000000,-835305986168751/5000000000000000])

In [0]:
x = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y = np.array([[1, 0], [1, 0], [1, 0], [0, 1]])
red = Neural_Net([2, 4, 3, 3, 2], [Logistica(), Logistica(), Logistica(), Logistica(), Logistica()])

#red.calculaDeltas(red.predict([0,0]), 1)

red.train(x,y)
print(red.predict([0,0]))
print(red.predict([1,0]))
print(red.predict([0,1]))
print(red.predict([1,1]))
#print(red.errors_[-1])

In [0]:
print(list(range(0, len([1,2,3,4,5]), 1)))

[0, 1, 2, 3, 4]
